### Group 92 Project Proposal
- Members: Alan, Bowen, Jia Yi, Kiki

# Predicting Heart Disease with Patient Health Information

## Introduction

Can we use the health report information to predict whether a patient has heart disease?

Heart disease is defined as the buildup of plaque in the heart's arteries that could lead to a heart attack/failure, which leads to death in many cases.

As the second leading cause of death in Canada, about 1 in 12 Canadian adults aged 20 and older live with a diagnosed heart disease, and the death rate of a heart disease is 2.9 times higher for them versus those without. However, early detection and management can help keep this illness under control. Hence, this project aims to build a classification model that can predict and diagnose whether a patient has a heart disease so that early intervention can take place to increase chances of rehabilitation.

The dataset chosen dates back to 1988 and consists of 4 databases from Cleveland, Hungary, Switzerland, and Long Beach V. The "target" field refers to the presence of heart disease in the patient. The integer valued ‘0’ means there is no disease, while the integer valued ‘1’ shows presence of heart disease.

## Methods & Data Analysis

We load the necessary libraries and adjust the options so that tables only print a maximum of 10 rows.

In [1]:
library(tidyverse)
library(tidymodels)                
library("ggplot2")  
options(repr.matrix.max.rows = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### Reading and Wrangling Data

#### The heart dataset has the following columns in the following order:
- Age - in years
- Sex -  (1= male; 0=female)
- Chest Pain Type - (0= typical angina; 1= atypical angina;  2= non-anginal pain; 3= asymptomatic
- Resting Blood Pressure - resting blood pressure (in mm mercury on first visit to hospital)
- Serum Cholestoral - in mg/dl
- Fasting Blood Sugar - (1= greater or equal to than 120 mg/dl; 0= less than 120 mg/dl
- Resting Electrocardiographic Results - (0 = normal; 1= having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV); 2= showing probable or definite left ventricular hypertrophy by Estes' criteria)
- Maximum Heart Rate Achieved - Maximum heart rate achieved in beats per minute
- Exercise Induced Angina - (1= yes; 0=no)
- Oldpeak - ST depression induced by exercise relative to rest
- Slope - (0= upsloping; 1= flat; 2= downsloping)
- Number of Major Vessels 
- Thal -  (3 = normal; 6 = fixed defect; 7 = reversable defect)
- Target - (0= undiagnosed; 1= diagnosed)

Alan: description of reading, cleaning, and wrangling the data

- use na.omit() to remove NA values in the dataset, but there are none in the first place
- removed sex, chest pain type, fasting blood sugar, resting electrocardiographic results, exercise induced angina, slope, number of major vessels, and thal variables
- we removed the variables that are either irrelevant to heart disease analysis or are actually a type, not numeric

In [2]:
#read data
heart <- read_csv("https://raw.githubusercontent.com/bowencu/dsci100groupproject/main/heart.csv")

#clean and wrangle data
heart <- heart |>
    mutate(target = as_factor(target)) |>
    select(-sex, -cp, -fbs, -restecg, -exang, -slope, -ca, -thal) |>
    na.omit()
heart


Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,trestbps,chol,thalach,oldpeak,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
52,125,212,168,1.0,0
53,140,203,155,3.1,0
70,145,174,125,2.6,0
61,148,203,161,0.0,0
62,138,294,106,1.9,0
⋮,⋮,⋮,⋮,⋮,⋮
59,140,221,164,0.0,1
60,125,258,141,2.8,0
47,110,275,118,1.0,0


- set the seed here to have reproducible results since splitting the data with random samples is a randomized process
- split the original dataframe into two sets, 75% for training and 25% for testing
- we split the data to construct a model from the training set, then test model on a dataset that hasn't been seen before to obtain a reasonable prediction accuracy of the model

In [3]:
set.seed(1)
heart_split <- initial_split(heart, prop = 0.75, strata = target)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

In [4]:
#data summarized in table, number of observations in each class of target
#0 meaning no heart disease, 1 meaning heart disease
summarize_heart <- heart_train |>
    group_by(target) |>
    summarize(n = n())
summarize_heart
print("Table 1.1: Summary of counts of heart disease cases")

target,n
<fct>,<int>
0,374
1,394


[1] "Table 1.1: Summary of counts of heart disease cases"


Table 1.1 summarizes the number of observations for each "target" classification. The "target" column is the classifier we are trying to predict where 0's represent no heart disease and 1's represent presence of heart disease. There appears to be roughly the same number of heart disease and no heart disease cases. This makes our analysis easier as the weighting of each classification is roughly balanced.

- 768 total observations
- 374 subjects without heart disease (~49% of total cases)
- 394 subjects with heart disease (~51% of total cases)

In [5]:
predictor_means <- heart_train |>
    map_df(mean, na.rm = TRUE) |>
    mutate(target = c("all cases"))
predictor_means
print("Table 1.2: Summary of the mean of each predictor")

no_disease_predictor_means <- heart_train |>
    filter(target == 0) |>
    map_df(mean, na.rm = TRUE) |>
    mutate(target = c("no heart disease"))
no_disease_predictor_means
print("Table 1.3: Means of each predictor in no heart disease cases")

disease_predictor_means <- heart_train |>
    filter(target == 1) |>
    map_df(mean, na.rm = TRUE) |>
    mutate(target = c("heart disease"))
disease_predictor_means
print("Table 1.4: Means of each predictor in heart disease cases")
   

Warning message in mean.default(.x[[i]], ...):
“argument is not numeric or logical: returning NA”


age,trestbps,chol,thalach,oldpeak,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
54.54688,131.1172,245.8099,149.2878,1.094141,all cases


[1] "Table 1.2: Summary of the mean of each predictor"


Warning message in mean.default(.x[[i]], ...):
“argument is not numeric or logical: returning NA”


age,trestbps,chol,thalach,oldpeak,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
56.79144,134.1818,250.7674,139.7059,1.643316,no heart disease


[1] "Table 1.3: Means of each predictor in no heart disease cases"


Warning message in mean.default(.x[[i]], ...):
“argument is not numeric or logical: returning NA”


age,trestbps,chol,thalach,oldpeak,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
52.41624,128.2081,241.1041,158.3832,0.5728426,heart disease


[1] "Table 1.4: Means of each predictor in heart disease cases"


In tables 1.2, 1.3, and 1.4, we found the means for each predictor for all cases, no heart disease cases, and heart disease cases. Comparing the means for each predictor in table 1.3 and table 1.4, the means do not differ significantly between the average heart disease and no heart disease case. It would be difficult selecting predictors to use for classification from these predictors as there are no strong distinguishing traits between the classifications.

### Methods

- Split the data into testing and training sets: 75% training and 25% testing
- Create a recipe, using columns “thalach” and “age” as predictors. Scale and centre predictors
- Create a validation set to determine the number of k-nearest-neighbors that yield the most accurate predictions. Perform a 5-fold cross validation on the training data
- Create k-nearest-neighbor model specification. Set ‘weight_func’ argument to rectangular to obtain the straight-line distance. Set ‘neighbors’ argument to tune(). Set engine to ‘kknn’ and set mode to ‘classification’ as we are predicting categorical variables
- Create a workflow combining the training data recipe and model. Use tune_grid() function and set the ‘grid’ argument equal to 10 K values. Use collect_metrics() to aggregate the mean and standard error
- Filter for accuracies from .metric column
- Create a visualisation to determine the best K value. Plot neighbors against accuracy estimate (mean).
- Build a model using the K value that provides the highest accuracy. Pass model and recipe through the fit() function.
- Use this final model to predict on the test dataset. Use predict() function and bind columns to the test dataset. 
- Report the accuracy of the prediction; collect metrics and filter .metric column for accuracy.
- Report the confusion matrix
- Create a new observation in a tibble. Pass knn model and new observation through predict() function to determine if the ‘new patient’ has heart disease

### Predictor Variable Selection - Kiki

- It is important to determine which predictors are irrelevant as these could otherwise negatively affect our classifier's performance; they tend to add a random amount of distance between each pair of observations, making the classifier less accurate.

- We will use the forward selection method to determine which predictors have a meaningful relationship to the target. This will include the following steps:

1. Start with a model having no predictiors.
2. Run the following 3 steps until you run out of predictors:
    1. For each unused predictor, add it to the    model to form a candidate model.
    2. Tune all of the candidate models.
    3. Update the model to be the candidate model with the highest cross-validation accuracy.
3. Select the model that provides the best trade-off between accuracy and simplicity.

In [ ]:
set.seed(1)

heart_subset <- heart_train |>
select(age, trestbps, chol, thalach, oldpeak, target)
names <- colnames(heart_subset |> select(-target))
# Create an empty tibble to store the results
accuracies <- tibble(size = integer(),
                     model_string = character(),
                     accuracy = numeric())

#create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
set_engine("kknn") |>
set_mode("classification")

#create a 5-fold cross-validation object
heart_vfold <- vfold_cv(heart_subset, v = 5, strata = target)

#store the total number of predictors
n_total <- length(names)

#stores selected predictors
selected <- c()

#for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    #for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        #create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("target", "~", paste(preds_new, collapse="+"))
        
        #create a recipe from the model string
        heart_recipe <- recipe(as.formula(model_string),
                               data = heart_subset) |>
        step_scale(all_predictors()) |>
        step_center(all_predictors())
        
        # tune the KNN classifier with these predictors, and collect the accuracy for the best K
        acc <- workflow() |>
        add_recipe(heart_recipe) |>
        add_model(knn_spec) |>
        tune_grid(resamples = heart_vfold, grid = 10) |>
        collect_metrics() |>
        filter(.metric == "accuracy") |>
        summarize(mx = max(mean))
        acc <- acc$mx |> unlist()
        
        #add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
        }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |>
    add_row(size = i,
            model_string = models[[jstar]],
            accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
    }
accuracies

### Visualizing the best predictors in the Training Data - Kiki

### Tuning the K-NN classifier

In [ ]:
#set seed
set.seed(1)

#update predcitors after Kiki's part
#create recipe
recipe_heart <- recipe(target ~ age + thalach, data = heart_train) |>
        step_scale(all_predictors()) |>
        step_center(all_predictors())

#parameter K value selection
#split training/validation sets
heart_vfold <- vfold_cv(heart_train, v = 5, strata = target)

#train classifier part 1 - create model specifications
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
       set_engine("kknn") |>
       set_mode("classification")

#create tibble with neighbors variable
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

#train classifier part 2 - create workflow
knn_results <- workflow() |>
        add_recipe(recipe_heart) |>
        add_model(knn_spec) |>
        tune_grid(resamples = heart_vfold, grid = k_vals) |>
        collect_metrics()

#find accuracy of the range of K values
accuracies <- knn_results |>
        filter(.metric == "accuracy")

#plot graph of accuracy against K value to find best K value
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
        geom_point() +
        geom_line() +
        geom_vline(xintercept = 1, colour = "red") +
        labs(x = "No. of Neighbors", 
             y = "Accuracy Estimate", 
             title = "Figure 1.1: Estimated Accuracies for range of K values") + 
        theme(text = element_text(size = 12))

accuracy_vs_k

We will now be tuning the classification model to find the best K value with the chosen predictors: X and Y

The following steps will be conducted:
1. Create a recipe, using columns “X” and “Y” as predictors. Scale and centre predictors.
2. Create a validation set to determine the number of k-nearest-neighbors that yield the most accurate predictions. Perform a 5-fold cross validation on the training data.
3. Create k-nearest-neighbor model specification. Set ‘weight_func’ argument to rectangular to obtain the straight-line distance. Set ‘neighbors’ argument to tune(). Set engine to ‘kknn’ and set mode to ‘classification’ as we are predicting categorical variables.
4. Create a workflow combining the training data recipe and model. Use tune_grid() function and set the ‘grid’ argument equal to 10 K values. Use collect_metrics() to aggregate the mean and standard error.
5. Filter for accuracies from .metric column.
6. Create a visualisation to determine the best K value. Plot neighbors against accuracy estimate (mean) to visualize the most stable k nearest neighbour.

As seen in Figure 1.1, k = 1 is the most suitable value for the number of neighbours since it has an estimated accuracy significantly higher than the other k values and does not have a big rise or drop in accuracy compared to the values one above or one below it.

### K-NN classification

In [ ]:
#create model specifications with best K value
knn_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 1) |>
        set_engine("kknn") |>
        set_mode("classification")

#create workflow
knn_fit <- workflow() |>
        add_recipe(recipe_heart) |>
        add_model(knn_spec_2) |>
        fit(data = heart_train)

#predict labels in testing dataset
heart_test_predictions <- predict(knn_fit, heart_test) |>
        bind_cols(heart_test) 

#compute classification accuracy
print("Figure 1.2: Accuracy of Predictions with K-NN Classification Model")
heart_test_predictions_2 <- heart_test_predictions |>
        metrics(truth = target, estimate = .pred_class) |>
        filter(.metric == "accuracy") |>
        select(.metric, .estimate) 
heart_test_predictions_2

#create confusion matrix
print("Figure 1.3: Confusion Matrix of Prediction Results of K-NN Classification Model")
confusion <- heart_test_predictions |>
        conf_mat(truth = target, estimate = .pred_class)
confusion

#create visualization from prediction outcome of model
plot <- heart_test_predictions |>
        ggplot(aes(x = .pred_class, fill = target)) +
        geom_bar(position = "fill") +
        labs(x = "Predicted Class", y = "Percentage of Observations in %",
            fill = "Actual Class") +
        ggtitle("Figure 1.4: Accuracy of Predictions of K-NN Classification Model") +
        scale_fill_discrete(labels = c("Has No Heart Disease", "Has Heart Disease")) +
        scale_y_continuous(labels = scales::label_percent(), 
                           breaks = scales::pretty_breaks(n = 10),
                           ) +
        scale_x_discrete(labels=c("0" = "Has No Heart Disease", "1" = "Has Heart Disease")) +
        theme(text = element_text(size = 16))
plot

* After we carried out K-NN classification on the testing data set, it was found that the accuracy of the trained model was at a high level of **97%**, as seen in Figure 1.2.
* We visualized the results via a confusion matrix as seen in Figure 1.3. From the matrix, it was observed that there was a total of 251 accurate predictions out of a total of 257 predictions. There were 6 cases where the actual class was "Has No Heart Disease" but the model predicted them as "Has Heart Disease".
* From Figure 1.4, we can hence see that 95% of predicted classes for "Has No Heart Disease" is accurate, while 100% of predicted classes for "Has Heart Disease" is accurate..

## Expected outcomes and significance

Heart disease patients need an immediate diagnosis.Doctors identify it based on results of clinical testing and prior experience treating patients presented with comparable symptoms. Our model will attempt to simulate a similar role, where age and maximum BPM are used to predict whether someone has heart disease. Furthermore, we expect to use a testing set to confirm the accuracy of our model. However, our current graph observes that there is a mix of points around the top middle area of the graph. We predict our classifier to have a lower accuracy for certain ranges of the data, which will cause serious health risks when misdiagnosing patients.
  
Limitations: Despite our data set having a handful of predictors that are numerical, we only use 2 that we believe are the biggest predictors. The investigation can be potentially extended by testing multiple predictors to find which one has the biggest impact. This would justify the selection of our 2 predictors from a statistical perspective. Furthermore, there is the potential to use 3 predictors at the same time to make a 3D model. However this would be hard to implement with our group’s current skills.

## References

https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset